DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml import DatasetBag, ExecutionConfiguration, Workflow, MLVocab
from deriva_ml.demo_catalog import create_demo_catalog, DemoML
import itertools
from IPython.display import display, Markdown
import pandas as pd

ImportError: cannot import name 'DerivaML' from partially initialized module 'deriva_ml.deriva_ml_base' (most likely due to a circular import) (/Users/carl/Repos/Projects/deriva-ml/src/deriva_ml/deriva_ml_base.py)

Set the details for the catalog we want and authenticate to the server if needed.

In [ ]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


Create a test catalog and get an instance of the DerivaML class.

In [ ]:
test_catalog = create_demo_catalog(hostname, domain_schema, create_features=True, create_datasets=True)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

In [ ]:
display(
    [f'{f.target_table.name}:{f.feature_name}' for f in ml_instance.find_features("Subject")],
    [f'{f.target_table.name}:{f.feature_name}' for f in ml_instance.find_features("Image")]
)

In [ ]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']
datasets = pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)
training_dataset_rid = [ds['RID'] for ds in ml_instance.find_datasets() if 'Training' in ds['Dataset_Type']][0]

display(
    Markdown(f'Training Dataset: {training_dataset_rid}'),
    Markdown('## Datasets'),
    datasets)

A feature is a set of values that are attached to a table in the DerivaML catalog. Instances of features are distingushed from one another by the ID of the execution that produced the feature value. The execution could be the result of a program, or it could be a manual process by which a person defines a set of values

To create a new feature, we need to know the name of the feature, the table to which it is attached, and the set of values that make up the feature.  The values could be terms from a controlled vocabulary, a set of one or more file based assets, or other values, such as integers, or strings. However, use of strings outside of controlled vocabularies is discouraged.

In [ ]:
ml_instance.add_term(MLVocab.workflow_type, "Manual Workflow", description="A Workflow that uses Deriva ML API")
ml_instance.add_term(MLVocab.execution_asset_type, "API_Model", description="Model for our API workflow")

api_workflow = Workflow(
    name="Manual Workflow",
    url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml",
    workflow_type="Manual Workflow",
    description="A manual operation"
)

manual_execution = ml_instance.create_execution(ExecutionConfiguration( description="Sample Execution", workflow=api_workflow))

# Now lets create model configuration for our program.
model_file = manual_execution.execution_assets_path('API_Model') / 'modelfile.txt'
with builtins.open(model_file, "w") as fp:
    fp.write(f"My model")

# Now upload the file and retrieve the RID of the new asset from the returned results.
uploaded_assets = manual_execution.upload_execution_outputs()
training_model_rid = uploaded_assets['API_Model/modelfile.txt'].result['RID']

Now create a dataset with our new assets.

Now we can add some features to our images.  To streamline the creation of new feature, we create a class that is specific to the arguments required to create it.

Now using TestFeatureClass, we can create some instances of the feature and add it.  We must have a exeuction_rid in order to define the feature.

In [ ]:
ml_instance.add_term(MLVocab.workflow_type, "API Workflow", description="A Workflow that uses Deriva ML API")
ml_instance.add_term(MLVocab.execution_asset_type, "API_Model", description="Model for our API workflow")

api_workflow = Workflow(
    name="API Workflow",
    url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml",
    workflow_type="API Workflow",
    description="A workflow that uses Deriva ML"
)

config = ExecutionConfiguration(
    datasets=[training_dataset_rid],
    assets = [training_model_rid],
    description="Sample Execution",
    workflow=api_workflow
)

ml_execution = ml_instance.create_execution(config)

In [ ]:
with MLExecute(ml_execution):
    # Get the input datasets:
    training_dataset = DatasetBag(ml_execution.bag_paths[0])  # Input dataset
    image_rids = training_dataset.get_table_as_dataframe('Image')['RID']

    # Get input files
    with open(ml_execution.asset_paths[0], 'rt') as model_file:
        training_model = model_file.read()
        print(f'Got model file: {training_model}')

    # Put your ML code here....
    pass

    # Write a new model
    model_file = manual_execution.execution_assets_path('API_Model') / 'modelfile.txt'
    with open(model_file) as f:
        f.write("Hello there a new model;\n")

    # Create some new feature values.
    bb_csv_path, bb_asset_paths = ml_execution.feature_paths('Image', 'BoundingBox')
    bounding_box_files = [bb_asset_paths['BoundingBox'] / f"box{i}.txt" for i in range(10)]
    for i in range(10):
        bounding_box_files.append(fn := bb_asset_paths['BoundingBox'] / f"box{i}.txt")
        with builtins.open(fn, "w") as fp:
            fp.write(f"Hi there {i}")

    ImageBoundingboxFeature = ml_instance.feature_record_class("Image", "BoundingBox")
    image_bounding_box_feature_list = [ImageBoundingboxFeature(Image=image_rid,
                                                               Execution=ml_execution.execution_rid,
                                                               BoundingBox=asset_rid)
                                       for image_rid, asset_rid in zip(image_rids, itertools.cycle(bounding_box_files))]

    ml_execution.write_feature_file(image_bounding_box_feature_list)

upload_status = ml_execution.upload_execution_outputs()

In [ ]:
e = (list(ml_instance.pathBuilder.deriva_ml.Execution.entities().fetch()))[0]

In [ ]:
pd.DataFrame(ml_instance.list_feature_values("Image", "Feature1")])

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)